import the libraries

In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

from sklearn.model_selection import train_test_split

from scipy.stats import skew

from sklearn.model_selection import cross_validate
import warnings
warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (7,4)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:.3f}'.format

Load the dataset 

In [2]:
df=pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.600,0.627,50,1
1,1,85,66,29,0,26.600,0.351,31,0
2,8,183,64,0,0,23.300,0.672,32,1
3,1,89,66,23,94,28.100,0.167,21,0
4,0,137,40,35,168,43.100,2.288,33,1


In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.isnull().sum()


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

feature_columns = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'
]

X = df[feature_columns]
y = df.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()
bagging_clf = BaggingClassifier(base_estimator=tree, n_estimators=1500, random_state=42)
bagging_clf.fit(X_train, y_train)

evaluate(bagging_clf, X_train, X_test, y_train, y_test)

TRAINIG RESULTS: 
CONFUSION MATRIX:
[[349   0]
 [  0 188]]
ACCURACY SCORE:
1.0000
CLASSIFICATION REPORT:
                0       1  accuracy  macro avg  weighted avg
precision   1.000   1.000     1.000      1.000         1.000
recall      1.000   1.000     1.000      1.000         1.000
f1-score    1.000   1.000     1.000      1.000         1.000
support   349.000 188.000     1.000    537.000       537.000
TESTING RESULTS: 
CONFUSION MATRIX:
[[117  34]
 [ 25  55]]
ACCURACY SCORE:
0.7446
CLASSIFICATION REPORT:
                0      1  accuracy  macro avg  weighted avg
precision   0.824  0.618     0.745      0.721         0.753
recall      0.775  0.688     0.745      0.731         0.745
f1-score    0.799  0.651     0.745      0.725         0.747
support   151.000 80.000     0.745    231.000       231.000


Creating sub samples to train models



In [8]:
#create random sub samples to train multiple models 
import sklearn
from sklearn import model_selection

from sklearn.model_selection import KFold

seed=42
Kfold=model_selection.KFold(n_splits=10, random_state=seed, shuffle=True)

Define a decision tree

In [9]:
#desing a dection tree classifier 
import sklearn
#from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier


cart= DecisionTreeClassifier() 
#cart= RandomForestClassifier() 

num_trees=1500

Classification model for bagging



In [10]:
#create classification model for bagging 
import sklearn
from sklearn.ensemble import BaggingClassifier
model= BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

Train models and print their accuracy

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
res=model_selection.cross_val_score(model, X_train, y_train, cv=Kfold)

for i in range(len(res)):
    print("Model:" +str(i)+ " accuracy is "+str(res[i]))

Model:0 accuracy is 0.7592592592592593
Model:1 accuracy is 0.8333333333333334
Model:2 accuracy is 0.7777777777777778
Model:3 accuracy is 0.7407407407407407
Model:4 accuracy is 0.7407407407407407
Model:5 accuracy is 0.7777777777777778
Model:6 accuracy is 0.7592592592592593
Model:7 accuracy is 0.9056603773584906
Model:8 accuracy is 0.7169811320754716
Model:9 accuracy is 0.7547169811320755


Print the mean accuracy

In [13]:
print("Mean accuracy is : " + str(res.mean()))

Mean accuracy is : 0.7766247379454927


Display the best model’s accuracy 

In [14]:
model.fit(X_train, y_train)
pred_label=model.predict(X_test)
nnz=np.shape(y_test)[0]- np.count_nonzero(pred_label-y_test)
acc=100*nnz/np.shape(y_test)[0]
print("Accuarcy is : " + str(acc))

Accuarcy is : 74.45887445887446
